In [1]:
!wget --no-check http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

--2025-03-07 21:13:45--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.165.196.48, 18.165.196.154, 18.165.196.132, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.165.196.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: ‘bm_preproc.py’

bm_preproc.py       100%[===================>]   9.18K  --.-KB/s    in 0.001s  

2025-03-07 21:13:45 (12.4 MB/s) - ‘bm_preproc.py’ saved [9400/9400]



In [2]:
!wget --no-check http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2025-03-07 21:14:06--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.165.196.48, 18.165.196.154, 18.165.196.132, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.165.196.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  3.13MB/s    in 0.2s    

2025-03-07 21:14:07 (3.13 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



Implement versions of the naive exact matching and Boyer-Moore algorithms that additionally:

* count and return the number of character comparisons performed
* count and return the number of alignments tried

Roughly speaking, these measure how much work the two different algorithms are doing.

In [3]:
from bm_preproc import BoyerMoore

In [29]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

def naive_with_counts(p, t):
    occurrences = []
    n_alignments = 0
    n_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        n_alignments += 1
        match = True
        for j in range(len(p)):  # loop over characters
            n_character_comparisons += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    print(f'''
occurrences: {occurrences}
alignments : {n_alignments}
comparisons: {n_character_comparisons}
''')
    return occurrences, n_alignments, n_character_comparisons


In [30]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    n_alignments = 0
    n_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        n_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            n_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    print(f'''
occurrences: {occurrences}
alignments : {n_alignments}
comparisons: {n_character_comparisons}
''')
    return occurrences, n_alignments, n_character_comparisons

## Naive Example 1

In [15]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


## Naive Example 2

In [16]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


## Boyer-Moore Example 1

In [11]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


## Boyer-Moore Example 2

In [12]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


## Q1 & Q2

In [21]:
from naive import readGenome

t = readGenome('chr1.GRCh38.excerpt.fasta')
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [32]:
_, _, _ = naive_with_counts(p, t)


occurrences: [56922]
alignments : 799954
comparisons: 984143



## Q3

In [35]:
p_bm = BoyerMoore(p, 'ATCG')
_, _, _ = boyer_moore_with_counts(p, p_bm, t)


occurrences: [56922]
alignments : 127974
comparisons: 165191

